## CulturaBot - création d'un chatbot sur la culture générale basé sur RAG destiné pour les ados/jeunes adultes

Le CulturaBot sera un chatbot qui utilisera les connaissances provenant du livre "La Culture générale pour les nuls" en version PDF pour répondre aux questions de culture générale de manière ludique et sans prétention. Ce projet se basera sur un modèle LLM extistant qui sera "augmenté" ou enrichi des connaissances dudit livre.

ETAPE 3 : Création du pipeline RAG

Le présent notebook a pour objectif d'utiliser la base de données vectorielles créée précédemment pour récupérer des données pertinentes relatives à la question posée par l'utilisateur ("retriever") afin de les ajouter à la question de l'utilisater ("augment"), les passer au LLM choisi et adapté à la langue française afin de ce dernier génére une réponse ("generator".)

In [1]:
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

c:\Users\aheij\Desktop\Jedha\Data Full Stack\Projects for Submission\06 Final Project bloc 6\projet-culture-g\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 1. Charger l'index FAISS
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
)
db = FAISS.load_local(
    "../data/faiss_index",
    embeddings,
    allow_dangerous_deserialization=True # Permet la désérialisation (à utiliser avec précaution)
)

In [5]:
# 2. Configurer TinyLlama pour le CPU
model_name = "../models/TinyLlama-1.1B-Chat-v1.0"  # Modèle optimisé pour le CPU, sauvegardé localement
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    device_map="cpu",  # Force l'utilisation du CPU
    load_in_8bit=False,  # Désactive pour le CPU (8bit est pour GPU)
    dtype="auto"
)

In [6]:
# 3. Pipeline optimisé pour le CPU
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=150,  # Réduit pour le CPU
    temperature=0.7,
    top_p=0.9,
    repetition_penalty=1.1
)
llm = HuggingFacePipeline(pipeline=pipe)

Device set to use cpu
C:\Users\aheij\AppData\Local\Temp\ipykernel_6300\3086219353.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [10]:
# 4. Prompt optimisé
template = """
Tu es un **prof de culture G ultra-cool** qui explique les trucs **comme à des potes de 16 ans**.
**Règles absolues :**
1. Réponds en **3 phrases max** (court et percutant).
2. Utilise **des emojis** (🗼, 💥, 😂, etc.) pour illustrer.
3. Fais **des comparaisons modernes** (ex: "comme un story Instagram", "comme un TikTok viral").
4. Évite **les termes compliqués** (remplace par des mots simples).
5. Si tu ne sais pas, dis-le avec humour ("Là, je sèche… comme un dev sans café ! ☕").

**Contexte (extraits du livre) :**
{context}

**Question du user :**
{question}

**Réponse (style ado, max 3 phrases, avec emojis) :**
"""
prompt = PromptTemplate(input_variables=["context", "question"], template=template)

In [11]:
# 5. Chaîne RAG
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=db.as_retriever(search_kwargs={"k": 3}),
    chain_type_kwargs={"prompt": prompt}
)

In [ ]:
6. # Test: poser une première question

question = "Pourquoi la Tour Eiffel a été construite ?"
response = qa_chain.invoke({"query": question})
print("🤖 Réponse :", response["result"])


🤖 Réponse : 
Tu es un **prof de culture G ultra-cool** qui explique les trucs **comme à des potes de 16 ans**.
**Règles absolues :**
1. Réponds en **3 phrases max** (court et percutant).
2. Utilise **des emojis** (🗼, 💥, 😂, etc.) pour illustrer.
3. Fais **des comparaisons modernes** (ex: "comme un story Instagram", "comme un TikTok viral").
4. Évite **les termes compliqués** (remplace par des mots simples).
5. Si tu ne sais pas, dis-le avec humour ("Là, je sèche… comme un dev sans café ! ☕").

**Contexte (extraits du livre) :**
carré, sont à Arras, Évreux et Milan. En pays normand apparaît ledonjon, monumentale tour carrée qui domine le paysage et sert autant des intentions de défense que dhabitation. Un des plus anciens exemples conservés est la Tour de Londres, construite de 1070 à 1090. CULTUREG.indb 245 091109 1009 DDeeuuxxiièèmmee ppaarrttiiee  PPlluuss bbeellllee llaa vviiee  aarrttss eett lliittttéérraattuurree En 1088, labbé Hugues de Cluny 1024-1109, À Clairvaux, les directives

In [ ]:
# Fonction pour poser des questions et afficher les résultats

def poser_question(question):
    response = qa_chain({"query": question})
    print("\n🔍 **Question :**", question)
    print("\n📄 **Chunks sélectionnés :**")
    if "source_documents" in response:
        print("\n📄 **Chunks sélectionnés :**")
        for i, doc in enumerate(response["source_documents"]):
            print(f"   {i+1}. {doc.page_content[:100]}...")
    else:
        print("\n📄 Aucun chunk source trouvé.")
    print("🤖 Réponse :", response["result"])
    return response


In [21]:
# Liste de questions à tester
questions = [
    "Pourquoi la Tour Eiffel a été construite ?",
    "C'est quoi la photosynthèse ?",
    "Qui était Napoléon ?",
    "Explique-moi la Révolution française en 3 lignes.",
    "Pourquoi la mer est salée ?"
]

# Pose toutes les questions
for q in questions:
    poser_question(q)
    print("\n" + "="*80 + "\n")



🔍 **Question :** Pourquoi la Tour Eiffel a été construite ?

📄 **Chunks sélectionnés :**

📄 Aucun chunk source trouvé.
🤖 Réponse : 
Tu es un **prof de culture G ultra-cool** qui explique les trucs **comme à des potes de 16 ans**.
**Règles absolues :**
1. Réponds en **3 phrases max** (court et percutant).
2. Utilise **des emojis** (🗼, 💥, 😂, etc.) pour illustrer.
3. Fais **des comparaisons modernes** (ex: "comme un story Instagram", "comme un TikTok viral").
4. Évite **les termes compliqués** (remplace par des mots simples).
5. Si tu ne sais pas, dis-le avec humour ("Là, je sèche… comme un dev sans café ! ☕").

**Contexte (extraits du livre) :**
carré, sont à Arras, Évreux et Milan. En pays normand apparaît ledonjon, monumentale tour carrée qui domine le paysage et sert autant des intentions de défense que dhabitation. Un des plus anciens exemples conservés est la Tour de Londres, construite de 1070 à 1090. CULTUREG.indb 245 091109 1009 DDeeuuxxiièèmmee ppaarrttiiee  PPlluuss bbeellllee